In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

%matplotlib inline

In [ ]:
precincts = gpd.read_file('../data/Police Precinct District Outlines (GIS).geojson')
precincts

In [ ]:
precincts.plot()

In [ ]:
incidents = gpd.read_file('../data/mnpd_incidents.geojson')

In [ ]:
incidents = incidents[incidents.geometry.notnull()]

In [ ]:
incidents[['longitude']] = incidents[['longitude']].apply(pd.to_numeric)  
incidents.info()

In [ ]:
incidents_clean = incidents[incidents.longitude < -10]
incidents_clean.sort_values('longitude', ascending = False)
incidents_clean.head()

In [ ]:
incidents_clean = incidents_clean.drop(columns = ['victim_county_resident', 'zip_code', 'victim_number', 'offense_nibrs', 
                                              'rpa', 'victim_race', 'incident_number', 'investigation_status', 
                                              'offense_number', 'domestic_related', 'victim_ethnicity', 
                                              'incident_location', 'victim_description', 'location_code', 'report_type', 
                                              'location_description', 'report_type_description', 'victim_type', 
                                              'incident_status_code', 'incident_status_description', 'zone', 'victim_gender'])
incidents_clean.info()

In [ ]:
incidents_clean['year'] = pd.DatetimeIndex(incidents_clean['incident_occurred']).year
incidents_clean['month'] = pd.DatetimeIndex(incidents_clean['incident_occurred']).month
incidents_clean.info()

In [ ]:
incbyprec = gpd.sjoin(incidents_clean, precincts, op = 'within')
incbyprec.year.value_counts()

In [ ]:
madinc = incbyprec.loc[incbyprec['precinct'] == 'MADISON']
madinc.info()

In [ ]:
mad1920 = madinc[madinc['year'].isin([2019, 2020])]
mad1920.shape

In [ ]:
madison_polygon = precincts[precincts.precinct == 'MADISON']
madison_polygon

In [ ]:
madcenter = madison_polygon.centroid.values[0]
area_madcenter = [madcenter.y, madcenter.x]
area_madcenter

In [ ]:
cluster_madmap = folium.Map(location =  area_madcenter, zoom_start = 11)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(cluster_madmap)

folium.GeoJson(madison_polygon).add_to(cluster_madmap)

# inside the loop add each marker to the cluster
for row_index, row_values in mad1920.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop1 = str(row_values['offense_description']) 
    pop2 = str(row_values['incident_occurred'])
    icon=folium.Icon(color="red",icon= 'exclamation-triangle', prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop1 + pop2,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
cluster_madmap.save('../maps/clustermadison.html')

cluster_madmap

In [ ]:
incidents2020 = incbyprec[incbyprec.year == 2020]
incjune2020 = incidents2020.loc[incidents2020['month'] == 6]
incjune2020.shape

In [ ]:
incjune2020.precinct.value_counts()

In [ ]:
#finding the precinct center
preccenter = precincts.centroid.values[0]

In [ ]:
area_preccenter = [preccenter.y, preccenter.x]
area_preccenter

In [ ]:
incjune2020.offense_description.value_counts().head(20)

In [ ]:
top3 = incjune2020.loc[incjune2020.offense_description.isin(['SIMPLE ASSLT', 'BURGLARY - MOTOR VEHICLE', 'DAMAGE PROP - PRIVATE'])]
top3.shape

In [ ]:

legend_html =   '''

                <div style="     position: fixed; 
                                 bottom: 25px; 
                                 left: 25px; 
                                 width: 250px; 
                                 height: 125px; 
                                 background-color: white;
                                 opacity: .85;
                                 border:2px solid grey; 
                                 z-index:9999;
     
     font-size:14px;
     font-weight: bold;
                >
                  &nbsp; <h5 style="text-align:center;">Incident Types</h5><br><br>
                  &nbsp; Simple Assault  &nbsp; <i class="fa fa-hand-rock-o" style="color:red"></i><br>
                  &nbsp; Burglary - Motor Vehicle  &nbsp; <i class="fa fa-car" style="color:green"></i><br>
                  &nbsp; Damage of Private Property  &nbsp; <i class="fa fa-frown-o" style="color:blue"></i><br>
                  
                </div>

                '''

top3map = folium.Map(location =  area_preccenter, zoom_start = 9)

marker_cluster = MarkerCluster().add_to(top3map)

folium.GeoJson(precincts, style_function = lambda x: {'fillColor': '#06bee1', 
                                                        'fillOpacity': .2, 
                                                        'color': '#03256c'}
              ).add_to(top3map)



# inside the loop add each marker to the cluster
for row_index, row_values in top3.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop2 = str(row_values['incident_occurred'])
    if row_values['offense_description'] == 'SIMPLE ASSLT':
        icon=folium.Icon(color='red',icon='hand-rock-o', prefix='fa')
    elif row_values['offense_description'] == 'BURGLARY - MOTOR VEHICLE':
        icon=folium.Icon(color='green',icon='car', prefix='fa')
    else:
        icon=folium.Icon(color='blue',icon='frown-o', prefix='fa')
    if row_values['offense_description'] == 'SIMPLE ASSLT':
        pop1 = 'Simple Assault'
    elif row_values['offense_description'] == 'BURGLARY - MOTOR VEHICLE':
        pop1 = 'Stolen Car'
    else:
        pop1 = 'Damage of Private Property'
        
    marker = folium.Marker(
        location = loc, 
        popup = pop1 + '<br/>' + '<br/>'+ pop2,
    icon = icon) 
    
    marker.add_to(marker_cluster)


#save an interactive HTML map by calling .save()
top3map.save('../maps/top3incidents.html')
top3map.get_root().html.add_child(folium.Element(legend_html))
top3map 